# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
from langchain_community.llms import Ollama
from langchain.embeddings import OllamaEmbeddings

llm = Ollama(model = "llama3")
embedding = OllamaEmbeddings(model="nomic-embed-text")

In [10]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL

In [13]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [18]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = False)

In [19]:
agent("What is the 25% of 300?")

{'input': 'What is the 25% of 300?', 'output': '**75.0**'}

In [21]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 

/Users/iqbal/Iqbal/Study/Deeplearning.ai/Langchain/.venv/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/iqbal/Iqbal/Study/Deeplearning.ai/Langchain/.venv/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/Users/iqbal/Iqbal/Study/Deeplearning.ai/Langchain/.venv/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser").

## Python Agent

In [22]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [24]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [25]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:Ollama] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, deb

Python REPL can execute arbitrary code. Use with caution.


[llm/end] [chain:AgentExecutor > chain:LLMChain > llm:Ollama] [5.27s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "I can solve this problem by using Python's built-in list sorting feature. I will first create a function to sort the customers by last name and then first name.\n\nAction: Python_REPL\nAction Input: \nPython REPL: >>> customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']",
        "generation_info": {
          "model": "llama3",
          "created_at": "2024-05-18T09:49:01.950325Z",
          "response": "",
          "done": true,
          "done_reason": "stop",
          "context": [
            128006,
            882,
            128007,
            198,
            198,
            2675,
            527,
            459,
            8479,
            6319,
            311,
            3350,
            323,
            9203,
           

'Agent stopped due to iteration limit or time limit.'

#### View detailed outputs of the chains

In [26]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:Ollama] Entering LLM run with input:
{
  "prompts": [
    "You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, deb

## Define your own tool

In [27]:
from langchain.agents import tool
from datetime import date

In [28]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [29]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [30]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Question: What's the date today?

Thought: To determine the current date, I'll use the "time" tool.

Action:
```json
{"action": "time", "action_input": ""}
```


Observation: 2024-05-18
Thought:I'm glad we're starting fresh!

Question: What's the date today?

Thought: To determine the current date, I'll use the "time" tool.

Action:
```json
{"action": "time", "action_input": ""}
```


Observation: 2024-05-18
Thought:I see what's going on here!

Final Answer: The date today is 2024-05-18.

> Finished chain.
